In [1]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from google.cloud import bigquery

C:\Users\Grethel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CREDS = r"D:\Datasets_proyecto\service_account.json"
client = bigquery.Client.from_service_account_json(json_credentials_path=CREDS)

# Configura tu proyecto y credenciales
project_id = 'windy-tiger-410421'

# Crea el cliente de BigQuery utilizando las credenciales proporcionadas
client = bigquery.Client(project=project_id, credentials=client._credentials)

# Especifica tu conjunto de datos y tabla para Yelp Reviews y Google Reviews
dataset_id = 'UltaBeautyReviews'
table_idY = 'yelp_reviews_ulta_beauty'
table_idG = 'google_reviews_ulta_beauty'

# Obtiene el esquema de la tabla de Yelp Reviews y Google Reviews
tableY = client.get_table(f'{project_id}.{dataset_id}.{table_idY}')
tableG = client.get_table(f'{project_id}.{dataset_id}.{table_idG}')

# Construye y ejecuta la consulta para Yelp Reviews
queryY = f"SELECT * FROM `{project_id}.{dataset_id}.{table_idY}` LIMIT 5"
query_jobY = client.query(queryY)
df_Y_ulta_beauty=query_jobY.to_dataframe()

# Construye y ejecuta la consulta para Google Reviews
queryG = f"SELECT * FROM `{project_id}.{dataset_id}.{table_idG}` LIMIT 5"
query_jobG = client.query(queryG)
df_G_ulta_beauty=query_jobG.to_dataframe()

print ('Datos cargados correctamente')



Datos cargados correctamente


In [3]:
ulta_beauty = pd.concat([df_G_ulta_beauty, df_Y_ulta_beauty], ignore_index=True)

In [4]:
ulta_beauty = ulta_beauty.dropna(subset=['text'])
ulta_beauty = ulta_beauty[ulta_beauty['text'].apply(lambda x: isinstance(x, str))]

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("Kaludi/Reviews-Sentiment-Analysis", use_auth_token=False)
tokenizer = AutoTokenizer.from_pretrained("Kaludi/Reviews-Sentiment-Analysis", use_auth_token=False)

C:\Users\Grethel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\models\auto\auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
C:\Users\Grethel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\models\auto\tokenization_auto.py:690: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [6]:
def analyze_sentiment(review):
    inputs = tokenizer(review, return_tensors="pt")
    outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    return predicted_label

# Apply the function to the 'text' column of ulta_beauty
ulta_beauty['sentiment'] = ulta_beauty['text'].apply(lambda x: analyze_sentiment(x))
ulta_beauty['sentiment_label'] = ulta_beauty['sentiment'].apply(lambda x: 'positive' if x == 1 else 'negative')
print(ulta_beauty['sentiment'])
print('El modelo se entreno correctamente')

0    0
1    1
2    1
3    1
4    1
5    0
6    1
7    0
8    0
9    0
Name: sentiment, dtype: int64
El modelo se entreno correctamente
